In [ ]:
import os
os.environ.setdefault('AWS_PROFILE', 'ghactivity')

In [ ]:
!aws s3 ls

In [ ]:
!aws s3 mb s3://aigithub

In [ ]:
!aws s3 rm s3://aigithub/ --recursive

In [ ]:
import boto3

dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('jobs')
table.delete_item(Key={'job_id': 'ghactivity_ingest'})
item = {
    'job_id': 'ghactivity_ingest',
    'job_description': 'Ingest ghactivity data to s3',
    'is_active': 'Y',
    'baseline_days': 3
}
table.put_item(Item=item)

In [ ]:
dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('jobs')

baseline_days = table. \
    get_item(Key={'job_id': 'ghactivity_ingest'})['Item']['baseline_days']

In [ ]:
baseline_days

In [ ]:
int(baseline_days)

In [ ]:
from datetime import datetime as dt
from datetime import timedelta as td

next_file = f'{dt.strftime(dt.now().date() - td(days=int(baseline_days)), "%Y-%m-%d")}-0.json.gz'

In [ ]:
next_file

In [ ]:
import requests
res = requests.get(f'https://data.gharchive.org/{next_file}')

In [ ]:
s3_client = boto3.client('s3')

In [ ]:
type(s3_client)

In [ ]:
s3_client.put_object?

In [ ]:
bucket_name = 'aigithub'

In [ ]:
type(res.content)

In [ ]:
upload_res = s3_client.put_object(
   Bucket=bucket_name,
   Key=f'landing/ghactivity/{next_file}',
   Body=res.content
)

In [ ]:
upload_res

In [ ]:
!aws s3 ls s3://aigithub/landing/ghactivity/

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_json(
    f's3://{bucket_name}/landing/ghactivity/{next_file}',
    lines=True,
    orient='records'
)

In [ ]:
df.shape

In [ ]:
df.count()

In [ ]:
df